In [1]:
# coding: utf-8

import random
import pickle
import numpy as np

random.seed(1234)

with open('../raw_data/reviews.pkl', 'rb') as f:
  reviews_df = pickle.load(f)
  reviews_df = reviews_df[['reviewerID', 'asin', 'unixReviewTime']]
    

with open('../raw_data/meta.pkl', 'rb') as f:
  meta_df = pickle.load(f)

  # 丢了大部分字段，只保留了两个字段
  meta_df = meta_df[['asin', 'categories']]



In [2]:
# 这是什么操作呢？？？
meta_df2 = meta_df
meta_df2['categories'] = meta_df2['categories'].map(lambda x: x[-1][-1])

In [3]:
def build_map(df, col_name):
  '''
  对df做修改——label encode；返回映射关系和key集合
  '''
  key = sorted(df[col_name].unique().tolist())
  # 这是做了labelEncode啊
  m = dict(zip(key, range(len(key))))
  df[col_name] = df[col_name].map(lambda x: m[x])
  return m, key

asin_map, asin_key = build_map(meta_df2, 'asin')
cate_map, cate_key = build_map(meta_df2, 'categories')
revi_map, revi_key = build_map(reviews_df, 'reviewerID')

In [4]:
meta_df2

,asin,categories
0,0,738
1,1,157
2,2,571
3,3,707
4,7,799
...,...,...
62996,62997,368
62997,62998,63
62998,62996,475
62999,62999,674


In [5]:
user_count, item_count, cate_count, example_count =\
    len(revi_map), len(asin_map), len(cate_map), reviews_df.shape[0]
print('user_count: %d\titem_count: %d\tcate_count: %d\texample_count: %d' %
      (user_count, item_count, cate_count, example_count))

user_count: 192403	item_count: 63001	cate_count: 801	example_count: 1689188


In [6]:
# 根据产品的labelEncode进行排序，这样的目的是什么？
meta_df3 = meta_df2.sort_values('asin').reset_index(drop=True)

In [7]:
reviews_df['asin'] = reviews_df['asin'].map(lambda x: asin_map[x])

In [8]:
reviews_df2 = reviews_df

In [9]:
# 也排序
reviews_df3 = reviews_df2.sort_values(['reviewerID', 'unixReviewTime']) \
                .reset_index(drop=True)


In [10]:
# 只保留了3个字段
reviews_df3 = reviews_df3[['reviewerID', 'asin', 'unixReviewTime']]

In [11]:
reviews_df3

,reviewerID,asin,unixReviewTime
0,0,13179,1400457600
1,0,17993,1400457600
2,0,28326,1400457600
3,0,29247,1400457600
4,0,62275,1400457600
...,...,...,...
1689183,192402,57576,1389744000
1689184,192402,22519,1396396800
1689185,192402,20977,1404172800
1689186,192402,60283,1404172800


In [12]:
meta_df3['categories']

0        738
1        157
2        571
3        707
4        714
        ... 
62996    475
62997    368
62998     63
62999    674
63000    351
Name: categories, Length: 63001, dtype: int64

In [13]:
meta_df3['categories'].values

array([738, 157, 571, ...,  63, 674, 351])

In [23]:
len(asin_map), max(asin_map.values()), min(asin_map.values()),meta_df3.shape

(63001, 63000, 0, (63001, 2))

### index是asin，value是cate的

In [24]:
cate_list = meta_df3['categories'].values

array([738, 157, 571, ...,  63, 674, 351])

In [25]:
meta_df3

,asin,categories
0,0,738
1,1,157
2,2,571
3,3,707
4,4,714
...,...,...
62996,62996,475
62997,62997,368
62998,62998,63
62999,62999,674


In [16]:
# 存下来，下个环节用
with open('../raw_data/remap.pkl', 'wb') as f:
  pickle.dump(reviews_df, f, pickle.HIGHEST_PROTOCOL) # uid, iid
  pickle.dump(cate_list, f, pickle.HIGHEST_PROTOCOL) # cid of iid line
  pickle.dump((user_count, item_count, cate_count, example_count),
              f, pickle.HIGHEST_PROTOCOL)
  pickle.dump((asin_key, cate_key, revi_key), f, pickle.HIGHEST_PROTOCOL)